In [49]:
import os

In [50]:
%pwd

'e:\\Projects\\kidney disease classification DL project'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\Projects\\kidney disease classification DL project'

In [51]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/belikenikhil03/Kidney-Disease-Classification-Deep-Learning-Project.mlflow"
os.environ["MLFLOW_TRACKING_TOKEN"] = "0266fe59012ad9be751415839370ac5a3a6bfcb7"

In [52]:

import tensorflow as tf

In [53]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [54]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [55]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [56]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/archive (1)/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/belikenikhil03/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [57]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import os
import dagshub
import mlflow
from mlflow.utils.mlflow_tags import MLFLOW_RUN_NAME
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [58]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

        dagshub.init(repo_owner='belikenikhil03',
                     repo_name='Kidney-Disease-Classification-Deep-Learning-Project',
                     mlflow=True)
    
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [59]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-07-27 16:25:34,222: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-27 16:25:34,225: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-27 16:25:34,227: INFO: common: created directory at: artifacts]


Initialized MLflow to track repo "belikenikhil03/Kidney-Disease-Classification-Deep-Learning-Project"

[2024-07-27 16:25:35,458: INFO: helpers: Initialized MLflow to track repo "belikenikhil03/Kidney-Disease-Classification-Deep-Learning-Project"]


Repository belikenikhil03/Kidney-Disease-Classification-Deep-Learning-Project initialized!

[2024-07-27 16:25:35,460: INFO: helpers: Repository belikenikhil03/Kidney-Disease-Classification-Deep-Learning-Project initialized!]
Found 1200 images belonging to 4 classes.
75/75 [==============================] - 259s 3s/step - loss: 15.1270 - accuracy: 0.2958
[2024-07-27 16:29:55,140: INFO: common: json file saved at: scores.json]


2024/07/27 16:29:58 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-07-27 16:30:02,106: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\PAYALA~1\AppData\Local\Temp\tmpfetno1j4\model\data\model\assets
[2024-07-27 16:30:04,976: INFO: builder_impl: Assets written to: C:\Users\PAYALA~1\AppData\Local\Temp\tmpfetno1j4\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/07/27 16:30:58 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.


In [48]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     